# Completion (02nd October 2023)

- Intake24 empty nutrition code ("N/A") -> Currently, filtered them out.
- Intake24 unmatched codes ("8416") -> Currently, left them out of calculations. (Helped answer a question)
- Improved efficiency in reading of all three files (Intake24: from 20s to almost 1s)
- Calculation(s) on non-recipes done (Example: shown in notebook)
- Skipping on food group: water
- Breakdown of granular recipes done

# Clarifications

- Which one to take: energy with fibre or energy without fibre? (applies for both intake24 and HEIFA)

- What is "Energy, with dietary fibre (kJ) per 100g"? How is it different from serving size and serving measure?

- Is it okay to drop rows where the Nutrient ID is N/A?

- What about cases when the Nutrient ID doesn't match? (Example: 8416)


# Questions to Ask

- Current output is: Print output. Anything specific you'd want to see?
- How do we breakdown the ingredients, which are recipes and consists of recipes?

For example: 

- Nutrition ID ABC (from Intake 24) has portion size of 150g
- It belongs to recipe D, which has 5 components: E F G H I
- One of the components I, has proportion of 0.60
- However I is a recipe itself.
- So: should we use the proportion of I for calculation or break it down further?

**Former**: 0.60 * 150 \
**Latter**: use sub-components and multiply with 150g

Intuitively and logically: 
- It makes sense to use the sub-components (latter) as it gives a more granular and accurate representation.
- It is consistent with the other calculations as well (the ones that don't have recipes themselves but are ingredients).

In [ ]:
from tqdm import tqdm

from User import User
from Survey import Survey
from Intake import Intake

from utils import *

import pandas as pd

import nest_asyncio
import asyncio

# Only run nest_asyncio in a Jupyter Notebook environment
nest_asyncio.apply()

In [ ]:
# Load the respective files

async def get_all_dataframes():
    return await asyncio.gather(
        load_intake24(),
        load_heifa_ingredients(),
        load_heifa_recipes()
    )

intake24_df, heifa_food_df, heifa_recipes_df = asyncio.run(get_all_dataframes())

# Breakdown of Intake 24:

The file has many users.

Each user has many surveys.

Each survey has many meal intake.

Each intake consists of many food components.

Every food component is marked with a "Nutrition ID code".

In [ ]:
user_dict = create_user_objects(intake24_df)

for user_id in user_dict.keys():

    print(f"Printing for User {user_id}")
    user_obj = user_dict[user_id]
    user_obj.print_information()

# Breakdown of HEIFA (Food Composition)

Every row in the file is a unique ingredient.

Every ingredient:
- has it's own attributes.
- can be mapped to a 8-digit code (for HEIFA Recipe)
- is used as a divisor for either energy (kilo joules) or grams (g)

In [ ]:
# Create the objects
food_composition_dict = create_food_objects(heifa_food_df)

for key, food_comp_obj in food_composition_dict.items():

    food_comp_obj.print_full_details()

# Breakdown of HEIFA (Recipes)

- Every recipe has multiple ingredients
- Keys are repeated across rows (similar to Survey ID of Intake24)
- Every ingredient has respective proportion to the recipe

In [ ]:
recipe_dict = create_recipe_objects(heifa_recipes_df)

for id, recipe_obj in recipe_dict.items():
    print(f"Printing for ID {id}\n")
    recipe_obj.print_ingredients_information()

## Mapping between Intake24 and HEIFA Ingredients

- For each user, extract the given nutrients and store in an array.
- This is from ALL the survey data.
- We don't care about the order here.
- The array will contain a list of dictionaries/JSON.

In the array:

- Use the HEIFA ID (from user) to map to the HEIFA Ingredients' HEIFA ID.
- Check if a result is found or not.
- Check if it requires a recipe or not.

## Mapping between Intake24 and HEIFA Recipes

This is in case a recipe is found (The second step).

- For the given recipe, extract the given nutrients ID and proportion, store in an array.
- We don't care about the order here.
- The array will contain a list of dictionaries.

In the array:

- Use the HEIFA ID (from the recipes) to map the HEIFA Ingredients' HEIFA ID.
- Check the energy and serving size.

In [ ]:
user_meals = fetch_user_food_list(user_dict)

In [ ]:
for user_id, meals_list in user_meals.items():

    print(f"Printing for user {user_id}")
    calculate_portion_serving_heifa(meals_list, food_composition_dict, recipe_dict)
    break
    print("\n\n")